In [35]:
import numpy as np

In [36]:
np.set_printoptions(precision=4, threshold=15)

In [37]:
import matplotlib.pyplot as plt

In [38]:
import tensorflow as tf

In [39]:
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPool2D

# Custom model

In [40]:
from sklearn.datasets import load_iris

In [41]:
from tensorflow.keras.utils import to_categorical

In [42]:
X, y = load_iris(return_X_y=True)
y = (y == 2) - 0

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [45]:
print("X_train shape : ", X_train.shape)
print("X_test shape : ", X_test.shape)
print("y_train shape : ", y_train.shape)
print("y_test shape : ", y_test.shape)

X_train shape :  (112, 4)
X_test shape :  (38, 4)
y_train shape :  (112,)
y_test shape :  (38,)


In [62]:
import numpy as np
class Model:
    '''
    Create a Simple Neural Network Model
    '''
    def __init__(self, n_inputs):
        '''
        n_inputs : number of neurons in the input layer
        '''
        self.layers = [{'name' : 'input_layer', 'input_units' : n_inputs}]
        self.activation_functions = {
            'linear' : lambda x : x, 
            'relu' : lambda x : x * (x > 0), 
            'sigmoid' : lambda x : 1 / (1 + np.exp(-x)),
        }
        self.gradients=[]
        self.error = 'No error calculated since forward_pass has not been called yet'
        self.num_layers = 1
        self.__m = n_inputs
    def add_layer(self, n, activation='linear', w=None, b=None):
        '''
        n : number of hidden units in the added layer
        activation : function to calculate activation outputs
        w : weights (randomly initialised not provided)
        b : biases (initialised to zero if not provided)
        '''
        if self.num_layers > 1:
            self.layers[-1]['name'] = 'hidden_layer' + str(self.num_layers)
        if not w:
            w = np.random.randn(self.__m, n) 
        if not b:
            b = np.random.randn(n, 1) * 0.001
        self.layers.append({
            'name' : 'output_layer',
            'hidden_units' : n, 
            'weights' : w, 
            'bias' : b, 
            'activation' : activation
        })
        self.num_layers += 1
        self.__m = n
        
    def dense(self, x, w, b, activation):
        '''
        x : output from previous layer
        returns the activation outputs
        '''
        f = self.activation_functions[activation]
        z = np.dot(w.T, x)
        z = np.add(z, b)
        a = f(z)
        
        return np.round(a, 8)
    
    def forward_pass(self, x):
        self.layers[0]['outputs'] = x
        for layer in self.layers[1:]:
            w = layer['weights']
            b = layer['bias']
            activation = layer['activation']
            a = self.dense(x, w, b, activation)
            layer['outputs'] = a 
            x = a
        return a
    
    def backpropagation(self, y, a, alpha):
        def get_dA(layer):
            a = layer['outputs']
            if layer['activation'] == 'sigmoid':
                return a * (1 - a) 
            elif layer['activation'] == 'relu':
                return (a > 0) - 0
            else:
                return 1
        dz = a - y
        w = np.array([[1]])
        i = self.num_layers - 1
        while i > 0:
            da = get_dA(self.layers[i])
            dz = w @ dz * da
            a = self.layers[i - 1]['outputs']
            dw = a @ dz.T
            w = self.layers[i]['weights']
            print(self.layers[i]['bias'].shape, dz.shape)
            db = dz.mean(axis=-1)
            
            self.layers[i]['weights'] = w - alpha * dw
            self.layers[i]['bias'] = self.layers[i]['bias'] - alpha * db
            i -= 1
    
    def fit(self, x, y, epochs=1, batch_size=None, learning_rate=0.001):
        for i in range(epochs):
            a = self.forward_pass(x)
            J = 1 / 2 * np.mean(np.power(np.subtract(y, a), 2))
            print(f'Epoch {i + 1}/{epochs}\n {x.shape[-1]}/{x.shape[-1]} [====================] - loss = {J}')
            self.backpropagation(y, a, learning_rate)
    
    def evaluate(self, x, y):
        a = self.forward_pass(x)
        J = 1 / 2 * np.mean(np.power(np.subtract(y, a), 2))
        return J

In [63]:
model = Model(4)

In [64]:
model.add_layer(8, 'relu')
model.add_layer(4, 'relu')
model.add_layer(1, 'sigmoid')

In [65]:
model.fit(X_train.T, y_train, epochs=10)

Epoch 1/10
 112/112 [====================] - loss = 0.1798792972320125
(1, 1) (1, 112)
(4, 1) (4, 112)
(8, 1) (8, 112)


ValueError: operands could not be broadcast together with shapes (8,112) (8,8) 

In [25]:
model.evaluate(X_test.T, y_test)

0.2630394075238181

In [ ]:
model = tf.keras.Sequential([
        Input((4,)),
        Dense(8, activation='relu'),
        Dense(4, activation='relu'),
        Dense(1, activation='sigmoid')
])

In [17]:
model.summary()

AttributeError: 'Model' object has no attribute 'summary'

In [18]:
model.compile(optimizer='SGD', loss='mse')

AttributeError: 'Model' object has no attribute 'compile'

In [19]:
model.fit(X_train, y_train, epochs=10, batch_size=1)

ValueError: shapes (8,4) and (112,4) not aligned: 4 (dim 1) != 112 (dim 0)

In [20]:
model.evaluate(X_test, y_test, batch_size=1)

TypeError: evaluate() got an unexpected keyword argument 'batch_size'

In [ ]:
y_pred = model.predict(X_test[-1].reshape(1,-1))

In [ ]:
y_pred

In [ ]:
np.argmax(y_pred)

In [ ]:
y_test[-1]

In [ ]:
w, b = model.layers[0].get_weights()